# Flight Price Prediction Project

##### Part 1- Data Collection

In [1]:
#Importing the neccessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
import pandas as pd
import time
import datetime
import smtplib
from email.mime.multipart import MIMEMultipart

In [36]:
#Connecting to the driver

driver=webdriver.Chrome(ChromeDriverManager().install())

C:\Users\vaish\AppData\Local\Temp\ipykernel_252\2821805735.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(ChromeDriverManager().install())


In [27]:
#Setting ticket types paths
return_ticket="//a[@aria-controls='wizard-flight-tab-roundtrip']"
one_way_ticket="//a[@aria-controls='wizard-flight-tab-oneway']"
multi_ticket="//a[@aria-controls='wizard-flight-tab-multicity']"


In [28]:
#defining function to select the type of ticket
def ticket_chooser(ticket):
    try:
        ticket_type=driver.find_element(By.XPATH,ticket)
        ticket_type.click()
    except Exception as e:
        pass

In [29]:
#defining function to select the departure city
def dep_country_chooser(dep_country):
    fly_from=driver.find_element(By.XPATH,'//*[@id="wizard-flight-tab-oneway"]/div[2]/div[1]/div/div[1]/div/div/div/div[1]')
    fly_from.click()
    spare=driver.find_element(By.XPATH,'//*[@id="location-field-leg1-origin"]')
    spare.send_keys(dep_country)
    time.sleep(3)
    first_item=driver.find_element(By.XPATH,'//li[@data-index="0"]')
    first_item.click()

In [30]:
#defining function to select the arrival city
def arrival_country_chooser(arrival_country):
    fly_to=driver.find_element(By.XPATH,'//*[@id="wizard-flight-tab-oneway"]/div[2]/div[1]/div/div[2]/div/div/div/div[1]') 
    fly_to.click()
    spare=driver.find_element(By.XPATH,'//*[@id="location-field-leg1-destination"]')
    spare.send_keys(arrival_country)
    time.sleep(3)
    first_item=driver.find_element(By.XPATH,'//li[@data-stid="location-field-leg1-destination-result-item"][1]')
    first_item.click()

In [31]:
#defining function to select departure date
def dep_date_chooser(dep_date):
    all_dates=[]
    dep_date_button=driver.find_element(By.XPATH,'//button[@id="d1-btn"]')
    dep_date_button.click()
    
    all_dates=driver.find_elements(By.XPATH,"//tbody//td//button[@class='uitk-date-picker-day']")
    for i in all_dates:
        if i.get_attribute('aria-label')==dep_date:
            i.click()
        break
    button=driver.find_element(By.XPATH,"//button[@data-stid='apply-date-picker']")
    button.click()

In [32]:
#defining function to select the arrival date
def return_date_chooser(ret_date):
    return_date_button=driver.find_element(By.XPATH,"//div[@class='uitk-layout-flex-item uitk-layout-flex-item-max-width-half_width uitk-layout-flex-item-flex-basis-zero uitk-layout-flex-item-flex-grow-1 uitk-date-field-wrapper'][2]")
    return_date_button.click()
    all_dates=[]
    
    all_dates=driver.find_elements(By.XPATH,"//tbody//td//button[@class='uitk-date-picker-day']")
    for i in all_dates:
        if i.get_attribute('aria-label')==ret_date:
            i.click()
        break
    button=driver.find_element(By.XPATH,"//button[@data-stid='apply-date-picker']")
    button.click()
    
  

In [33]:
#Function to select the search button
def search():
    search=driver.find_element(By.XPATH,"//button[@class='uitk-button uitk-button-large uitk-button-fullWidth uitk-button-has-text uitk-button-primary']")
    search.click()
    time.sleep(15)
    print('Results ready!')

In [34]:
#Function to scrape flight ticket prices, departure-time, arrival time, airline and layover

df=pd.DataFrame()
def compile_data(date):
    global dep_times_list
    global arr_times_list
    global airlines_list
    global price_list
    global layover
    global layover_spare
    global layover_list
    global departure_city
    global arrival_city
    global total_time
    global tot_cities
    global spare4
    global df
    global date_list
    
    
    time.sleep(5)
    #Scraping price
    a=driver.find_elements(By.XPATH,"//span[@class='uitk-lockup-price']")
    price_list=[i.text for i in a]
      
    #Scraping airline name    
    b=driver.find_elements(By.XPATH,"//div[@data-test-id='flight-operated']")
    airlines_list=[i.text for i in b]
    
    #Scraping departure and arrival time
    total_time=driver.find_elements(By.XPATH,"//span[@data-test-id='departure-time']")
    total_time_list=[value.text for value in total_time]
    spare1=[]
    for i in total_time_list:
        spare1.append(i.split("-"))
        
    #departure times and arrival time
    dep_times_list=[spare1[i][0] for i in range(len(spare1))]
    arr_times_list=[spare1[i][1] for i in range(len(spare1))]
    
    
    #scraping departure city and arrival city
    tot_cities=driver.find_elements(By.XPATH,"//div[@data-test-id='arrival-departure']")
    spare3=[]
    spare3=[value.text for value in tot_cities]
    spare4=[]
    spare4=[value.split("-") for value in spare3]
    departure_city=[spare4[i][0] for i in range(len(spare4))]
    arrival_city=[spare4[i][1] for i in range(len(spare4))]
           
    #Scraping layover list
    e=driver.find_elements(By.XPATH,'//div[@data-test-id="journey-duration"]')
    spare5=[]
    spare5=[value.text.split("(") for value in e]
    layover=[spare5[i][1] for i in range(len(spare5))]
    layover_spare=[layover[i].split(")")  for i in range(len(layover))]
    layover_list=[layover_spare[i][0] for i in range(len(layover_spare))]
    
    #Scraping date of journey
    date_list=[date for i in range(len(price_list))]
    
    
    
    #creating data frame and exporting to csv format
    df=pd.DataFrame({"Departure_time":dep_times_list,"Arrival_time":arr_times_list,"Departure_city":departure_city,"Arrival_city":arrival_city,"Date_departure":date_list,"Airline":airlines_list,"Layover":layover_list,"Price":price_list})
    df.to_csv('expedia_5.csv', encoding='utf-8',header=False,mode='a')
    header=False

In [39]:
city_list=["Ahmedabad","Cuddapah","Rajahmundry","Vishakapatnam","Vadodara","Mangalore"]
date_list=["1 Feb 2023","2 Feb 2023","3 Feb 2023","4 Feb 2023","5 Feb 2023","6 Feb 2023","7 Feb 2023","8 Feb 2023","9 Feb 2023"]
for city in city_list:
    for value in date_list:
        link='https://www.expedia.co.in/'
        driver.get(link)
        time.sleep(2)
        #choose flights only
        flights_only=driver.find_element(By.XPATH,'//*[@id="wizardMainRegionV2"]/div/div/div/div/ul/li[2]')
        flights_only.click()
        ticket_chooser(one_way_ticket)
        dep_country_chooser("Hyderabad")
        arrival_country_chooser(city)
        dep_date_chooser(value)
        search()
        compile_data(value)

Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!
Results ready!


ElementClickInterceptedException: Message: element click intercepted: Element <button data-testid="submit-button" type="submit" class="uitk-button uitk-button-large uitk-button-fullWidth uitk-button-has-text uitk-button-primary">...</button> is not clickable at point (508, 364). Other element would receive the click: <button type="button" class="uitk-date-picker-day" data-day="11" aria-label="11 Feb 2023"></button>
  (Session info: chrome=108.0.5359.126)
Stacktrace:
Backtrace:
	(No symbol) [0x00ECF243]
	(No symbol) [0x00E57FD1]
	(No symbol) [0x00D4D04D]
	(No symbol) [0x00D828B9]
	(No symbol) [0x00D808CC]
	(No symbol) [0x00D7E4CB]
	(No symbol) [0x00D7D164]
	(No symbol) [0x00D732A6]
	(No symbol) [0x00D9858C]
	(No symbol) [0x00D72BFF]
	(No symbol) [0x00D98804]
	(No symbol) [0x00DAC9EB]
	(No symbol) [0x00D98386]
	(No symbol) [0x00D7163C]
	(No symbol) [0x00D7269D]
	GetHandleVerifier [0x01169A22+2655074]
	GetHandleVerifier [0x0115CA24+2601828]
	GetHandleVerifier [0x00F78C0A+619850]
	GetHandl